In [ ]:
import fxcmpy
import time
import datetime as dt
import talib as ta
import numpy as np

### STRATEGY DESCRIPTION ####
# This strategy buys when the Fast SMA crosses over the Slow SMA and sell when the Fast SMA crosses
# under the Slow SMA. Opposing signals will close out opposing positions if close_on_opposing_signal == True.
# If close_on_opposing_signal == False, the strategy can open hedged positions (long & short at the same time)
# Parameters allow traders to change token, symbol, timeframe, Fast SMA & Slow SMA Periods, 
# Opposing signal closing logic, trade size/stop/limit.
# This is a close-of-bar strategy, meaning it only signals trades at the close of a bar.
# For more strategy examples, please visit github.com/fxcm/RestAPI
#############################

###### USER PARAMETERS ######
# token = '7363fec3b64647198252257812994375c9769f4d'
symbol = 'EUR/USD'
timeframe = "m1"	        # (m1,m5,m15,m30,H1,H2,H3,H4,H6,H8,D1,W1,M1)
close_on_opposing_signal = True
amount = 1
stop = -10
trailstop_value = 10
limit = 50
#############################

# Global Variables
pricedata = None
numberofcandles = 300

# Connect to FXCM API
con = fxcmpy.fxcmpy(access_token="f7635348a30fe53b8766c07279290150a8e76b30", log_level="error")
# con = fxcmpy.fxcmpy(config_file='fxcm.cfg')
	
# This function runs once at the beginning of the strategy to run initial one-time processes/computations
def Prepare():
	global pricedata
	
	print("Requesting Initial Price Data...")
	pricedata = con.get_candles(symbol, period=timeframe, number=numberofcandles)
	print(pricedata)
	print("Initial Price Data Received...")

# Get latest close bar prices and run Update() function every close of bar/candle
def StrategyHeartBeat():
	while True:
		currenttime = dt.datetime.now()
		if timeframe == "m1" and currenttime.second == 0 and GetLatestPriceData():
			Update()
		elif timeframe == "m5" and currenttime.second == 0 and currenttime.minute % 5 == 0 and GetLatestPriceData(): 
			Update()
			time.sleep(240)
		elif timeframe == "m15" and currenttime.second == 0 and currenttime.minute % 15 == 0 and GetLatestPriceData(): 
			Update()
			time.sleep(840)
		elif timeframe == "m30" and currenttime.second == 0 and currenttime.minute % 30 == 0 and GetLatestPriceData():
			Update()
			time.sleep(1740)
		elif currenttime.second == 0 and currenttime.minute == 0 and GetLatestPriceData():
			Update()
			time.sleep(3540)
		time.sleep(1)

# Returns True when pricedata is properly updated			
def GetLatestPriceData():
	global pricedata
	
	# Normal operation will update pricedata on first attempt
	new_pricedata = con.get_candles(symbol, period=timeframe, number=numberofcandles)
	if new_pricedata.index.values[len(new_pricedata.index.values)-1] != pricedata.index.values[len(pricedata.index.values)-1]:
		pricedata= new_pricedata
		return True
		
	counter = 0
	# If data is not available on first attempt, try up to 3 times to update pricedata
	while new_pricedata.index.values[len(new_pricedata.index.values)-1] == pricedata.index.values[len(pricedata.index.values)-1] and counter < 3:
		print("No updated prices found, trying again in 10 seconds...")
		counter+=1
		time.sleep(10)
		new_pricedata = con.get_candles(symbol, period=timeframe, number=numberofcandles)
	if new_pricedata.index.values[len(new_pricedata.index.values)-1] != pricedata.index.values[len(pricedata.index.values)-1]:
		pricedata = new_pricedata
		return True
	else:
		return False
		
# This function is run every time a candle closes
def Update():
	print(str(dt.datetime.now()) + "	 " + timeframe + " Bar Closed - Running Update Function...")
    
    
	#inputs
	h_left = 10
	h_right = 10
	sample_period = 5000
	show_ptz = True
	show_channel = True
	colFinal ="none"

	closes = np.array(pricedata['bidclose'])
	highs = np.array(pricedata['bidhigh'])
	lows = np.array(pricedata['bidlow'])
    
	len_array = closes.size
	a = len_array-21
	closes_last10 = closes[a:len_array]
	high_last10 = highs[a:len_array]
	low_last10 = lows[a:len_array]

    
    
    #Arrows
	central_bar_low = pricedata['bidlow'][len(pricedata['bidlow'])-11]
	central_bar_high =  pricedata['bidhigh'][len(pricedata['bidhigh'])-11]
	full_zone_low = np.amin(low_last10)
	full_zone_high = np.amax(high_last10)
    
	if central_bar_high >= full_zone_high:
		colFinal = "lime"
	elif central_bar_low <= full_zone_low:
		colFinal = "maroon"


    #TRADING LOGIC
    #Buy If colFinal is aqua and colFinal2 is lime
	if colFinal == "lime":
		print("	  BUY SIGNAL!")
		if close_on_opposing_signal and countOpenTrades("S") > 0:
			print("	  Closing Sell Trade(s)...")
			exit("S")
		if countOpenTrades("B") == 0:
			print("	  Opening Buy Trade...")
			enter("B")
        
    #Sell If colFinal is orange and colFinal2 is red
	if colFinal == "maroon":
		print("	  SELL SIGNAL!")
		if close_on_opposing_signal and countOpenTrades("B") > 0:
			print("	  Closing Buy Trade(s)...")
			exit("B")
		if countOpenTrades("S") == 0:
			print("	  Opening Sell Trade...")
			enter("S")    
        
	
	# Print Price/Indicators
	print("Close Price: " + str(pricedata['bidclose'][len(pricedata)-1]))
	print("ColFinal: " + colFinal)
	print(central_bar_high)
	print(full_zone_high)
		
	print(str(dt.datetime.now()) + "	 " + timeframe + " Update Function Completed.\n")

# This function places a market order in the direction BuySell, "B" = Buy, "S" = Sell, uses symbol, amount, stop, limit
def enter(BuySell):
	direction = True;
	if BuySell == "S":
		direction = False;
	try:
		opentrade = con.open_trade(symbol=symbol, is_buy=direction,amount=amount, time_in_force='GTC',order_type='AtMarket',is_in_pips=True,limit=limit, stop=stop, trailing_step=trailstop_value)
	except:
		print("	  Error Opening Trade.")
	else:
		print("	  Trade Opened Successfully.")

# This function closes all positions that are in the direction BuySell, "B" = Close All Buy Positions, "S" = Close All Sell Positions, uses symbol
def exit(BuySell=None):
	openpositions = con.get_open_positions(kind='list')
	isbuy = True
	if BuySell == "S":
		isbuy = False
	for position in openpositions:
		if position['currency'] == symbol:
			if BuySell is None or position['isBuy'] == isbuy:
				print("	  Closing tradeID: " + position['tradeId'])
				try:
					closetrade = con.close_trade(trade_id=position['tradeId'], amount=position['amountK'])
				except:
					print("	  Error Closing Trade.")
				else:
					print("	  Trade Closed Successfully.")

# Returns number of Open Positions for symbol in the direction BuySell, returns total number of both Buy and Sell positions if no direction is specified
def countOpenTrades(BuySell=None):		
	openpositions = con.get_open_positions(kind='list')
	isbuy = True
	counter = 0
	if BuySell == "S":
		isbuy = False
	for position in openpositions:
		if position['currency'] == symbol:
			if BuySell is None or position['isBuy'] == isbuy:
				counter+=1
	return counter

Prepare() # Initialize strategy
StrategyHeartBeat() # Run strategy

Requesting Initial Price Data...
                     bidopen  bidclose  bidhigh   bidlow  askopen  askclose  \
date                                                                          
2020-03-17 11:35:00  1.10291   1.10290  1.10308  1.10272  1.10306   1.10304   
2020-03-17 11:36:00  1.10290   1.10305  1.10309  1.10287  1.10304   1.10319   
2020-03-17 11:37:00  1.10305   1.10284  1.10309  1.10281  1.10319   1.10300   
2020-03-17 11:38:00  1.10284   1.10232  1.10306  1.10232  1.10300   1.10245   
2020-03-17 11:39:00  1.10232   1.10189  1.10236  1.10179  1.10245   1.10203   
2020-03-17 11:40:00  1.10189   1.10177  1.10205  1.10162  1.10203   1.10192   
2020-03-17 11:41:00  1.10177   1.10178  1.10195  1.10161  1.10192   1.10191   
2020-03-17 11:42:00  1.10178   1.10173  1.10187  1.10164  1.10191   1.10186   
2020-03-17 11:43:00  1.10173   1.10196  1.10197  1.10160  1.10186   1.10210   
2020-03-17 11:44:00  1.10196   1.10195  1.10204  1.10179  1.10210   1.10209   
2020-03-17 11:45:00

2020-03-17 16:37:01.262610	 m1 Bar Closed - Running Update Function...
Close Price: 1.0979
ColFinal: none
1.09965
1.09986
2020-03-17 16:37:01.263641	 m1 Update Function Completed.

2020-03-17 16:38:01.080545	 m1 Bar Closed - Running Update Function...
	  BUY SIGNAL!
	  Opening Buy Trade...
	  Trade Opened Successfully.
Close Price: 1.09793
ColFinal: lime
1.09986
1.09986
2020-03-17 16:38:03.715194	 m1 Update Function Completed.

2020-03-17 16:39:01.585661	 m1 Bar Closed - Running Update Function...
Close Price: 1.09811
ColFinal: none
1.09978
1.09986
2020-03-17 16:39:01.586656	 m1 Update Function Completed.

2020-03-17 16:40:01.389626	 m1 Bar Closed - Running Update Function...
Close Price: 1.09782
ColFinal: none
1.09916
1.09986
2020-03-17 16:40:01.390623	 m1 Update Function Completed.

2020-03-17 16:41:01.220582	 m1 Bar Closed - Running Update Function...
Close Price: 1.09735
ColFinal: none
1.09853
1.09986
2020-03-17 16:41:01.221541	 m1 Update Function Completed.

2020-03-17 16:42:01.09